In [2]:
import sys
from pathlib import Path

def find_project_root(markers=["src", ".env"], max_hops=7):
    """Busca la raíz del proyecto por múltiples marcadores"""
    p = Path.cwd()
    for _ in range(max_hops):
        if any((p / marker).exists() for marker in markers):
            return p
        p = p.parent
    raise RuntimeError(f"No se encontró la raíz del proyecto desde {Path.cwd()}")

# Encontrar y añadir la raíz al path
PROJECT_ROOT = find_project_root()
if str(PROJECT_ROOT) not in sys.path:
    sys.path.insert(0, str(PROJECT_ROOT))

print(f"[OK] Raíz del proyecto: {PROJECT_ROOT}")
print(f"[OK] sys.path actualizado")

[OK] Raíz del proyecto: C:\Users\manue\OneDrive\Escritorio\Proyecto WhoScored
[OK] sys.path actualizado


In [3]:
from datetime import date
from pathlib import Path
# Importar después de configurar sys.path
from src.whoscored_viz.whoscored_fixtures import make_driver, scrape_range_finished
from src.whoscored_viz.paths import BASE_DATA_DIR  # Usar la ruta configurada

COMP_SLUG   = "laliga"
SEASON_SLUG = "2025-2026"

start_date = date(2025, 8, 1)
end_date   = date(2025, 8, 31)

# Usar BASE_DATA_DIR en lugar de PROJECT_ROOT
out_dir = BASE_DATA_DIR / "raw" / "fixtures"

driver = make_driver(use_uc=True, headless=False)
df_fixtures = scrape_range_finished(driver, start_date, end_date, out_dir,
                                    comp_slug=COMP_SLUG, season_slug=SEASON_SLUG, save_json=True, fixtures_url=None)
driver.quit()

df_fixtures.head()

[paths.py] PROJECT_ROOT: C:\Users\manue\OneDrive\Escritorio\Proyecto WhoScored
[paths.py] BASE_DATA_DIR: C:\Users\manue\OneDrive\Escritorio\Proyecto WhoScored\data


ImportError: cannot import name 'FIXTURES_DIR' from 'src.whoscored_viz.paths' (C:\Users\manue\OneDrive\Escritorio\Proyecto WhoScored\src\whoscored_viz\paths.py)

In [ ]:
# IMPORTA DESDE TU MÓDULO (editable o sys.path)
from datetime import date
from pathlib import Path
from src.whoscored_viz.paths import FIXTURES_DIR  # ← ruta absoluta a data/raw/fixtures
from src.whoscored_viz.whoscored_fixtures import make_driver, scrape_range_finished, FIXTURES_URL

# Comprobación de dónde vamos a escribir
out_dir = FIXTURES_DIR  # p.ej. C:\Users\manue\OneDrive\Escritorio\Proyecto WhoScored\data\raw\fixtures
print("[OUT] FIXTURES_DIR:", out_dir)
out_dir.mkdir(parents=True, exist_ok=True)

# Parámetros de scraping
COMP_SLUG   = "laliga"
SEASON_SLUG = "2025-2026"
start_date  = date(2025, 8, 1)
end_date    = date(2025, 8, 31)

# Verificación explícita de URL (evita el 'data;' en blanco)
print("FIXTURES_URL =", FIXTURES_URL)
assert isinstance(FIXTURES_URL, str) and FIXTURES_URL.startswith("http"), "FIXTURES_URL inválida"

# Crea driver (usa visible para depurar; cambia a headless=True cuando esté estable)
driver = make_driver(use_uc=True, headless=False)
try:
    df_new = scrape_range_finished(
        driver, start_date, end_date, out_dir,
        comp_slug=COMP_SLUG, season_slug=SEASON_SLUG,
        save_json=True,
        fixtures_url=FIXTURES_URL   # 👈 nunca None
    )
finally:
    driver.quit()

# Guardado consolidado defensivo (por si el scraper devolviera parciales)
df_all = consolidate_fixtures(df_new)
if df_all.empty:
    raise RuntimeError("No se obtuvieron fixtures. Revisa la navegación y el rango de fechas.")

write_safe_csv_json(df_all, save_json=True)
print("[OK] CSV consolidado →", out_dir / "DataFixtures" / COMP_SLUG / SEASON_SLUG / "finished_matches.csv")
print("[OK] Filas:", len(df_all))
display(df_all.head(10))